# Zpracování a prezentace dat

Notebook pro získání moderátorů a respondentů zastoupených ve vybraných pořadech vysílání Českého rozhlasu.  

## Knihovny

In [1]:
import csv
import math
import os

import pandas as pd
import requests
import cro.schedule as schedule
from pathlib import Path


DATA = Path("../data")

# import sys
#!{sys.executable} -m pip install scrapy
# import sys; sys.path
# import numpy; numpy.__path__

## Konfigurace

Od uživatele chceme aby zadal požadované období (dny) pro které budeme provádět analýzu.

### Vstup od uživatele

In [2]:
dates = [f"2023-04-{i}" for i in range(1, 30)]

print(dates)

['2023-04-1', '2023-04-2', '2023-04-3', '2023-04-4', '2023-04-5', '2023-04-6', '2023-04-7', '2023-04-8', '2023-04-9', '2023-04-10', '2023-04-11', '2023-04-12', '2023-04-13', '2023-04-14', '2023-04-15', '2023-04-16', '2023-04-17', '2023-04-18', '2023-04-19', '2023-04-20', '2023-04-21', '2023-04-22', '2023-04-23', '2023-04-24', '2023-04-25', '2023-04-26', '2023-04-27', '2023-04-28', '2023-04-29']


## Zpracování

### Získej program pro stanici Plus a Radiožurnál

In [3]:
client = schedule.Client("plus")
schedules_ps = list(client.get_day_schedule(date) for date in dates)

In [4]:
client = schedule.Client("radiozurnal")
schedules_rz = list(client.get_day_schedule(date) for date in dates)

In [5]:
schedules = schedules_rz + schedules_ps

for s in schedules:
    print(s)

Schedule(station=Radiožurnál, date=2023-04-01, shows=84)
Schedule(station=Radiožurnál, date=2023-04-02, shows=83)
Schedule(station=Radiožurnál, date=2023-04-03, shows=88)
Schedule(station=Radiožurnál, date=2023-04-04, shows=88)
Schedule(station=Radiožurnál, date=2023-04-05, shows=89)
Schedule(station=Radiožurnál, date=2023-04-06, shows=87)
Schedule(station=Radiožurnál, date=2023-04-07, shows=83)
Schedule(station=Radiožurnál, date=2023-04-08, shows=83)
Schedule(station=Radiožurnál, date=2023-04-09, shows=82)
Schedule(station=Radiožurnál, date=2023-04-10, shows=81)
Schedule(station=Radiožurnál, date=2023-04-11, shows=88)
Schedule(station=Radiožurnál, date=2023-04-12, shows=86)
Schedule(station=Radiožurnál, date=2023-04-13, shows=89)
Schedule(station=Radiožurnál, date=2023-04-14, shows=89)
Schedule(station=Radiožurnál, date=2023-04-15, shows=79)
Schedule(station=Radiožurnál, date=2023-04-16, shows=82)
Schedule(station=Radiožurnál, date=2023-04-17, shows=87)
Schedule(station=Radiožurnál, d

### Vyber jen požadované pořady

In [6]:
schedule_data = []

for schedule in schedules:
    for index, show in enumerate(schedule.shows):
        if len(show.moderators) == 0:
            moderators = ""
        else:
            moderators = ";".join(p.name for p in show.moderators)
        relation_type = "repríza" if show.repetition else "premiéra"

        line = dict(
            station_name=show.station.name,
            show_type=show.type,
            show_kind_name=show.kind.name,
            show_description=show.description,
            show_id=show.id,
            show_title=show.title,
            show_date=show.date,
            show_since=show.since,
            show_till=show.till,
            relation_type=relation_type,
            show_duration_minutes=math.ceil(show.duration / 60),
            moderators=moderators,
        )

        schedule_data.append(line)

### Ulož program do CSV pro další zpracování

In [7]:
# Uložíme CSV pro další zpracování.
with open(DATA / "schedule.csv", mode="w") as file:
    writer = csv.DictWriter(file, fieldnames=schedule_data[0].keys())
    writer.writeheader()
    writer.writerows(schedule_data)

---
**ZAČNI ZDE, POKUD CHCEŠ PRACOVAT S JIŽ ULOŽENÝM PROGRAMEM**

In [8]:
SHOWS_WANTED = (
    "Pro a proti",
    "Interview Plus",
    "Rozhovory a komentáře",
    "Dvacet minut Radiožurnálu",
    "Hlavní zprávy - rozhovory a komentáře",
    "Speciál",
)

### Načti program a získej premiéry

- Načteme data do tabulky a aplikujeme voodoo a černou magii.
  - Vybereme pouze premiéry pořadů.
- Respondenty získáme pomocí parsování a služby Geneea (*entity recognition*).


In [9]:
df = pd.read_csv(DATA / "schedule.csv")
df = df[df.relation_type == "premiéra"]  # FIXME

### Ukaž unikátní názvy pořadů 

In [10]:
shows_ps = set(df[df["station_name"] == "Plus"]["show_title"].to_list())
shows_rz = set(df[df["station_name"] == "Radiožurnál"]["show_title"].to_list())

print("POŘADY STANICE PLUS:\n", shows_ps)
print("POŘADY STANICE RADIOŽURNÁL:\n", shows_rz)

POŘADY STANICE PLUS:
 {'Týden Plus', 'Radiokniha', 'Zaostřeno', 'Za obzorem', 'Online Plus', 'Knížky Plus', 'Dvacet minut Radiožurnálu', 'Odpolední Plus', 'Pro a proti', 'Zprávy v angličtině/News in English', 'Peníze a vliv', 'Historie Plus', 'Bruselské chlebíčky', 'Osobnost Plus', 'Svět ve 20 minutách', 'Mezi námi', 'Hlavní zprávy - rozhovory, komentáře', 'Kultura Plus', 'Leonardo Plus', 'Chyba systému', 'RozhlaSTO', 'Názory a argumenty', 'Ranní Plus', 'Jak to bylo doopravdy', 'Vertikála', 'Portréty', 'Studio Leonardo', 'Téma Plus', 'Hovory', 'Ex libris', 'Souvislosti Plus', 'Příběhy 20. století', 'Řečí peněz', 'Natura', 'Dvacítka Radiožurnálu', 'Interview Plus', 'Zprávy', 'Život k nezaplacení', 'Archiv Plus', 'Reportáže zahraničních zpravodajů', 'Speciál', 'Dokument Plus', 'Hlavní zprávy', 'Vinohradská 12', 'Laboratoř', 'Zprávy v půl'}
POŘADY STANICE RADIOŽURNÁL:
 {'Vznik nové knihovny Milana Kundery v Brně', 'Dvacet minut Radiožurnálu', 'Noční Radiožurnál', 'Fotbal: AC Sparta Praha 

In [11]:
# Vyber pouze sledované pořady
df = df[df["show_title"].isin(SHOWS_WANTED)]  # FIXME
df.head(100)

,station_name,show_type,show_kind_name,show_description,show_id,show_title,show_date,show_since,show_till,relation_type,show_duration_minutes,moderators
45,Radiožurnál,AFTERNOON,Zpravodajství,"O všem, co se právě děje",18615603,Hlavní zprávy - rozhovory a komentáře,2023-04-01,12:09:47,12:30:00,premiéra,21,NaN
129,Radiožurnál,AFTERNOON,Zpravodajství,"O všem, co se právě děje",18615787,Hlavní zprávy - rozhovory a komentáře,2023-04-02,12:09:18,12:30:00,premiéra,21,NaN
215,Radiožurnál,AFTERNOON,Zpravodajství,"O všem, co se právě děje",18638357,Hlavní zprávy - rozhovory a komentáře,2023-04-03,12:10:00,12:30:00,premiéra,20,NaN
233,Radiožurnál,AFTERNOON,Publicistika,"Hostem je Hana Kordová Marvanová, právnička a ...",18638395,Dvacet minut Radiožurnálu,2023-04-03,17:06:00,17:30:00,premiéra,24,Tomáš Pancíř
236,Radiožurnál,EVENING,Zpravodajství,"O všem, co se právě děje",18638402,Hlavní zprávy - rozhovory a komentáře,2023-04-03,18:10:59,18:30:00,premiéra,20,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4076,Plus,AFTERNOON,Publicistika,"Hostem je Martin Pýcha, předseda Zemědělského ...",18604814,Dvacet minut Radiožurnálu,2023-04-19,17:06:00,17:30:00,premiéra,24,NaN
4130,Plus,MORNING,Publicistika,Ministr práce a sociálních věcí Marian Jurečka...,18604879,Pro a proti,2023-04-20,09:34:59,09:59:10,premiéra,25,Karolína Koubová
4138,Plus,NOON,Publicistika,"Pavel Klíma, poslanec z TOP09. Podle úpravy na...",18604891,Interview Plus,2023-04-20,11:33:31,12:00:00,premiéra,27,Jan Bumba
4159,Plus,AFTERNOON,Publicistika,"Hostem je Martin Krsek, historik a senátor. Zh...",18604922,Dvacet minut Radiožurnálu,2023-04-20,17:06:00,17:30:00,premiéra,24,NaN


## Kontrola

Zkontrolujeme si, jestli máme očekávaný počet premiér pořadů (nyní za měsíc):
- Dvacet minut Radiožurnálu: 23 (premiér/březen) * 2 (stanic) = 46 premiér / březen (23 na Radiožurnálu)
- Interview Plus: 23 (premiér/březen) * 1 (stanic) = 23 premiér / březen
- Pro a proti: 23 (premiér/březen) * 1 (stanic) = 23 premiér / březen
- Hlavní zprávy - rozhovory a komentáře: 23 * 2 (dvakrát za prac. den) * 2 (stanic) = 92 (46 na Radiožurnálu)


In [12]:
# Kolik máme řádků/premiér pořadů (epizod).
len(df)

118

In [13]:
#  Na Plusu se premiéry objevují jen pro dva dny v týdnu, zřejmě můžeme Plus úplně vyfiltrovat.
df = df[
    ~(
        (df["show_title"] == "Dvacet minut Radiožurnálu")
        & (df["station_name"] == "Plus")
    )
]
df = df[
    ~(
        (df["show_title"] == "Hlavní zprávy - rozhovory a komentáře")
        & (df["station_name"] == "Plus")
    )
]

print(
    "Očekávaný počet premiér 'Dvacet minut Radiožurnálu' odpovídá:",
    len(df[df["show_title"] == "Dvacet minut Radiožurnálu"]) == 23,
)
print(
    "Očekávaný počet premiér 'Interview Plus' odpovídá:",
    len(df[df["show_title"] == "Interview Plus"]) == 23,
)
print(
    "Očekávaný počet premiér 'Pro a proti' odpovídá:",
    len(df[df["show_title"] == "Pro a proti"]) == 23,
)
print(
    "Očekávaný počet premiér 'Hlavní zprávy - rozhovory a komentáře' odpovídá:",
    len(df[df["show_title"] == "Hlavní zprávy - rozhovory a komentáře"]) == 46,
    len(df[df["show_title"] == "Hlavní zprávy - rozhovory a komentáře"]),
)

# df[df["show_title"] ==  "Hlavní zprávy - rozhovory a komentáře" ].head(100)

Očekávaný počet premiér 'Dvacet minut Radiožurnálu' odpovídá: False
Očekávaný počet premiér 'Interview Plus' odpovídá: False
Očekávaný počet premiér 'Pro a proti' odpovídá: False
Očekávaný počet premiér 'Hlavní zprávy - rozhovory a komentáře' odpovídá: True 46


### Získej popis epizod pro NLP analýzu

In [14]:
index_description_df = df[["show_title", "show_description"]]

#### Ukázka záznamů

In [15]:
index_description_df.head(5)

,show_title,show_description
45,Hlavní zprávy - rozhovory a komentáře,"O všem, co se právě děje"
129,Hlavní zprávy - rozhovory a komentáře,"O všem, co se právě děje"
215,Hlavní zprávy - rozhovory a komentáře,"O všem, co se právě děje"
233,Dvacet minut Radiožurnálu,"Hostem je Hana Kordová Marvanová, právnička a ..."
236,Hlavní zprávy - rozhovory a komentáře,"O všem, co se právě děje"


#### Počet záznamů

In [16]:
len(index_description_df)

104

### Vytvoř objekty pro NLP analýzu

Po potřeby Geneea Media V2 API vytvoříme JSON objekty, které zasíláme v dotazu. Jako index (atribut `id`) použijeme index řádku datového rámce, ten pak dostaneme po zpracování zpět. Tím můžeme získané entity opět spojit s původními řádky datového rámce. Pro jeden analyzovaný text můžeme dostat 0 až N rozpoznaných osob. Nekteré z nich mohou být moderátoři, ty se snažíme odfiltrovat pomocí daného seznamu.

In [17]:
inputs = []
for i, record in index_description_df.iterrows():
    inputs.append(
        {
            "id": i,
            "paraSpecs": [{"type": "text", "text": record.show_description}],
        }
    )

# Uložíme si vstupy pro přehled.
# with open(DATA / "test.txt", mode="w", encoding="utf8") as f:
# f.write("\n".join([str(i) for i in inputs]))

In [18]:
BASE_URL = "https://media-api.geneea.com/v2/"
HEADERS = {"content-type": "application/json", "X-API-Key": os.getenv("GENEEA_API_KEY")}

results = [
    requests.post(f"{BASE_URL}nlp/analyze", json=input, headers=HEADERS).json()
    for input in inputs
]

In [19]:
len(results) == len(index_description_df)

True

### Uložíme  získané entity/osoby spolu s původním indexem záznamu 

Rozpoznaná entita má buď typ `person` nebo v některých případech typ `other`. To pak musíme hledat klíč `feats` a v něm dále `detectedType`. 

**Klíče nemusí existovat, proto testujeme jejich přítomnost!**

In [20]:
index_person = []
for result in results:
    if "entities" in result:
        for entity in result["entities"]:
            if entity["type"] == "person":
                index_person.append((result["id"], entity["stdForm"], entity["gkbId"]))
            if "feats" in entity:
                if "detectedType" == entity["feats"]:
                    if entity["feats"]["detectedType"] == "person":
                        index_person.append(
                            (result["id"], entity["stdForm"], entity["gkbId"])
                        )

In [21]:
respondents_df = pd.DataFrame(index_person, columns=["index", "respondent", "gkbid"])
respondents_df.set_index("index")
respondents_df.index.astype(int)
respondents_df

,index,respondent,gkbid
0,233,Hana Kordová Marvanová,G12018971
1,233,Petr Pavel,G2080040
2,233,Petr Pithart,G685676
3,322,Donald Trump,G22686
4,410,Hana Naiclerová,G77833405
...,...,...,...
84,4730,Lukáš Kovanda,G12034372
85,4730,Karolína Koubová,G58241908
86,4817,Petr Pavel,G2080040
87,4817,Kateřina Stojanová,G38df309d-E


In [22]:
# Vrátíme osoby do původního dataframu. Grupujeme si osoby podle indexu.
merged_respondents_df = respondents_df.groupby(["index"])
merged_respondents_df = merged_respondents_df["respondent"].apply(";".join)
merged_respondents_df.reset_index().set_index("index")
merged_respondents_df = pd.DataFrame(merged_respondents_df)
merged_respondents_df

,respondent
index,
1090,Petr Musil
1178,Věra Jourová;Emmanuel Macron;Vladimír Kroc
1426,Ondřej Kundra;Andrej Babiš;Vladimír Kroc
1515,Vladimír Kroc
1693,Martin Krsek
1782,Jan Bartošek;Andrej Babiš
2034,Josef Středula;Vladimír Kroc
2124,Robert Plaga;Vladimír Balaš;Vladimír Kroc
2212,Alexandr Vondra;Vladimír Kroc


In [23]:
merged_respondents_df.index = merged_respondents_df.index.astype(int)
print(merged_respondents_df.index.dtype)

# Iterace: Tohle je pěkná blbost z hlediska rychlosti, ale merge nějak nefungoval.
# Nicméně smyčka funguje a zjistili jsme, že indexy neměli správny typ, proto původní merge nefungoval.
# nechávám jako důkaz a připomínku vlastní blbosti.
# for row in df.iterrows():
#     if row[0] in merged_respondents_df.index:
#         print(row[0], merged_respondents_df.loc[row[0]])

int32


In [24]:
final_df = df.join(merged_respondents_df)

In [25]:
final_df.to_csv(DATA / "final.csv")

### Zjisti funkce osob pomocí Geneea

**Nyní nemám přístup k této funkcionalitě.**

Zjistíme funkce pro všechny rozpoznané osoby. Prozatím negunguje / access denied

In [26]:
ids = respondents_df["gkbid"].to_list()
data_input = {
    "ids": ids,
    "language": "cs",
}

result = requests.post(
    f"{BASE_URL}knowledgebase/infoboxes", json=data_input, headers=HEADERS
)
result.content

b'Access Denied'